## 로지스틱 회귀 주요 하이퍼파라미터 상세 설명

---

### 1. C (정규화 강도, Inverse of regularization strength)

- **정의**: 모델의 규제(패널티) 세기를 조절하는 반비례 상수  
  \[
    \min \; \frac{1}{n}\sum L(y, \hat y) \;+\;\frac{1}{2C}\|w\|^2
  \]
- **C 값이 작을수록**  
  - \(1/2C\) 패널티 항이 커져 **강한 규제**  
  - 계수(w)가 작아지고, 모델이 단순해져 **과소적합** 경향  
- **C 값이 클수록**  
  - 규제 약화 → 계수가 자유로워져 **과적합** 가능  
- **사용 팁**:  
  - 데이터가 **노이즈가 많거나** 특성 간 상관이 높으면 작은 C(강한 규제)  
  - 데이터가 **매우 깔끔하고** 특성 수가 적으면 큰 C 혹은 규제 없음(‘none’)

---

### 2. penalty (규제 유형)

- **‘l2’ (Ridge 유형)**  
  - 계수 제곱합을 패널티로 사용  
  - 모든 특성을 조금씩 중요도로 남기면서 과적합 억제  
- **‘l1’ (Lasso 유형)**  
  - 계수 절댓값 합을 패널티로 사용  
  - 일부 계수를 정확히 **0**으로 만들어 **특성 선택**  
- **‘elasticnet’** (scikit-learn 지원 시)  
  - L₁과 L₂를 혼합 (`l1_ratio` 추가 지정)  
- **‘none’**  
  - 규제를 적용하지 않음 (OLS 형태)

---

### 3. solver (최적화 알고리즘)

- **‘liblinear’**  
  - 소규모 데이터, L₁ 규제 지원  
  - 1대1(one-vs-rest) 방식으로 다중 클래스 해석  
- **‘lbfgs’**  
  - 준뉴턴 방법(Quasi-Newton)  
  - L₂ 규제만 지원, 대규모 데이터/다중 클래스에 적합  
- **‘saga’**  
  - 확률적 경사하강 기반 변형  
  - L₁, L₂, ElasticNet 모두 지원  
  - 대규모·희소 데이터에 강함  
- **‘newton-cg’** / **‘sag’** 등  
  - 각각 수치 특성에 맞춰 속도/메모리 효율 차이 있음

---

### 4. multi_class (다중 클래스 처리 방식)

- **‘ovr’ (One-vs-Rest)**  
  - 각 클래스를 양성-음성 문제로 분리하여 독립 학습  
  - 클래스 수 만큼 모델 생성  
- **‘multinomial’**  
  - 소프트맥스 함수를 활용해 다중 클래스 동시 최적화  
  - 로짓벡터 전체를 한 번에 학습 → 확률 분포 일관성 보장  
- **팁**: 클래스 수가 많지 않고 데이터가 균형적이면 `multinomial`,  
  불균형하거나 희소한 경우 OVR이 더 안정적일 수 있음.

---

### 5. max_iter (최대 반복 횟수)

- 최적화 수렴을 위해 solver가 반복할 최대 이터레이션 수  
- 너무 작으면 “수렴 실패” 경고 → 늘려야 함 (`max_iter=1000` 등)

---

### 6. tol (수렴 허용 오차)

- 반복 중 “손실 변화량”이 이 값 이하로 떨어지면 조기 종료  
- 작은 tol → 엄격한 수렴 (반복 많아짐), 큰 tol → 빠른 종료

---

> **실전 팁**  
> - `GridSearchCV`로 **C**, **penalty**, **solver** 조합을 탐색  
> - `saga` + `l1` (Lasso) → 자동 특성 선택  
> - `lbfgs` + `multinomial` → 다중 클래스에 부드러운 확률 예측  
> - `liblinear` + `l2` → 소규모·이진 분류에서 빠르고 안정적  


In [ ]:
**코드 셀 (Python)**
```python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score

# (예시) Iris 데이터셋 사용
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# 1) Pipeline 정의
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000))
])

# 2) 하이퍼파라미터 그리드
param_grid = {
    'clf__C': [0.01, 0.1, 1, 10],
    'clf__penalty': ['l2', 'l1'],
    'clf__solver': ['liblinear', 'saga'],
    'clf__multi_class': ['ovr', 'multinomial']
}

# 3) GridSearchCV 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
gs = GridSearchCV(
    pipe,
    param_grid,
    cv=cv,
    scoring='accuracy',
    n_jobs=-1
)

# 4) 학습 및 결과 확인
gs.fit(X_tr, y_tr)
print("Best parameters:", gs.best_params_)
print("Best cross-val accuracy:", gs.best_score_)

# 5) 테스트 세트 성능
best_model = gs.best_estimator_
y_pred = best_model.predict(X_te)
print("Test set accuracy:", accuracy_score(y_te, y_pred))

# SGD (Stochastic Gradient Descent) 사용 시기와 손실 함수 심화

---

## 1. SGD란 무엇인가?

- **경사하강법**(Gradient Descent)의 변형으로,  
  전체 데이터가 아니라 **미니배치**(또는 한 샘플) 단위로 기울기를 계산해 파라미터를 업데이트  
- 매 스텝마다 데이터의 일부만 보기 때문에  
  - **계산 비용**이 작고 메모리 효율적  
  - **노이즈**가 섞여 있어 지역 최적(local minima) 회피에 도움  

---

## 2. 언제 SGD를 사용하나?

1. **대규모 데이터셋**  
   - 전체 데이터를 한 번에 메모리에 올리기 힘들 때  
   - 배치 크기(batch size)를 조절해 GPU/CPU 자원 활용

2. **온라인 학습(Online Learning)**  
   - 새로운 데이터가 지속적으로 유입될 때  
   - 한 샘플씩 즉시 업데이트하며 모델을 지속 개선

3. **딥러닝/신경망**  
   - 모델 파라미터(가중치)가 수십만~수백만 개일 때  
   - 전체 그래디언트를 구하기엔 계산 비용이 너무 큼

4. **빠른 프로토타이핑**  
   - 빠른 수렴 경향을 이용해 초반 성능을 빠르게 확인

---

## 3. 손실 함수(Loss Function) 개념

- **목적**: 모델 예측  \(\hat y\)와 실제값 \(y\) 간 차이를 수치화  
- **최적화 목표**:  
  \[
    \min_\theta \; L(\theta)
  \]
  여기서 \(L\)이 손실 함수, \(\theta\)는 파라미터(가중치, 절편 등)

---

## 4. 대표적 손실 함수

| 문제 유형      | 손실 함수               |
| **회귀**      | MSE (Mean Squared Error)|
|             | MAE (Mean Absolute Error)|
| **이진 분류**  | Binary Cross-Entropy|
| **다중 분류**  | Categorical Cross-Entropy  |
| **히브리드**  | Huber Loss                 |

---

## 5. 손실 함수 선택 기준

1. **오차 특성**  
   - 이상치가 많으면 MAE/Huber  
   - 극단값 중점을 두면 MSE

2. **출력 형태**  
   - 확률 예측이 필요하면 Cross-Entropy

3. **연속 vs 범주형**  
   - 회귀: MSE/MAE  
   - 분류: Cross-Entropy

4. **수치 안정성**  
   - 로그 항이 들어가는 경우 `ε` 추가(클리핑, log-softmax)

---

> **요약**:  
> - **SGD**는 대용량·온라인·딥러닝에 특화된 최적화 기법  
> - **손실 함수**는 문제 유형과 데이터 특성에 맞춰 선택  
> - SGD와 손실 함수(미니배치) 조합으로 효율적이고 안정적인 학습이 가능  
